In [ ]:
from elasticsearch import Elasticsearch

import time

import pandas as pd
import plotly.express as px

In [ ]:
# Elastic host
es = Elasticsearch(
    hosts=[
            "https://localhost:9200"
    ],
    basic_auth=("elastic", "NES9DZ-QwhanXAQf9caV"),
#     use_ssl=True,
#     verify_certs=False,
    ca_certs="./ca.crt"
)

In [ ]:
# Word counts df
word_count_csv = 'C:/Users/Asher/Documents/School/_Scriptie/Data/xml_word_counts.csv'

df_word_count = pd.read_csv(word_count_csv)

display(df_word_count)

test_count = df_word_count[df_word_count["filename"] == 'ParlaMint-BE_2014-06-19-54-plenair-ip001x.xml']["words"].values[0]
print(test_count)

In [ ]:
# NOTE: input is {search_key: value, search_key2: value2}
# query function
def query(search_dict):
    processed_search_list = []
    
    # loop door alle search elements heen
    for k, v in search_dict.items():
        processed_search_list.append({"match_phrase" : {k : v}})
        
    # stel de uitkomst samen
    result = es.search(
    index = "search",
    size = 10000, # TODO: Zorg dat er een groter limit is dan 10000
    query = {
        "bool" : {
            "must": processed_search_list,
        },
    })
    
    return result

In [ ]:
# Oude functie waarmee ik een andere manier van Ngram maken testte op snelheid
# # NOTE: input is {search_key: value, search_key2: value2}

# # query function
# def query_termvectors(search_dict, countries=["BE"]): #, "BE", "FR"
#     # de zoekterm
#     term = search_dict["content_simplified"]
    
#     # return dataframe
#     df = pd.DataFrame({'date': [],
#                       'percentage': [],
#                       'country': []
#                       })
    
#     # doe een query per land
#     for country in countries:
#         search_dict["country"] = "BE"
    
#         query_output = query(search_dict)

#         # files, ids
#         files = {}
        
#         # dates, files
#         dates = {}

#         for res in query_output['hits']['hits']:
#             seg_id = res["_source"]["file"]
#             date = res["_source"]["year"] + res["_source"]["month"] + res["_source"]["day"]
            
#             if seg_id in files:
                
#                 files[seg_id] += [res["_id"]]
                
#             else:
                
#                 files[seg_id] = [res["_id"]]
                
#             if date in dates and seg_id not in dates[date]:
                
#                 dates[date] += [seg_id]
                
#             else:
                
#                 dates[date] = [seg_id]
        
#         # loop door alle files op een date heen
#         for date in dates:
#             files_per_date = dates[date]
#             print(date, files_per_date , "\n")
#             for file in files_per_date:
#                 print(date, file, "\n")
#                 file_term_freq = 0

#                 for ids in files[file]:
#                     print(ids)
#                     result = es.mtermvectors(
#                         ids=[ids], 
#                         index="search",
#                         term_statistics=False,
#                         field_statistics=False,
#                         offsets=False,
#                         payloads=False,
#                         positions=False,
#                         fields=["content_simplified"]
#                     )

#                     id_term_freq = result["docs"][0]["term_vectors"]["content_simplified"]["terms"][term]["term_freq"]
#                     file_term_freq = file_term_freq + int(id_term_freq)

#                 word_total = df_word_count[df_word_count["filename"] == file]["words"].values[0]    
# #                 print(file, file_term_freq, word_total)
    
#     return result

In [ ]:
# Oude testruimte
# start_time = time.time()

# test_term = "democratie"

# test_newgram = query_termvectors({"content_simplified":test_term})

# print("--- %s seconds ---" % (time.time() - start_time))
# print(test_newgram["docs"][0], "\n")

In [ ]:
# query testruimte
start_time = time.time()

# de query zelf
# test_result = query({"content_simplified":"ete", 
#                      "person_simplified":"FreDeriC BARBIER", 
#                      "country":"FR"})

# test_result = query({"content_simplified":"groen",
#                      "party":"SP",
#                      "year":"2018"})

test_result = query({"content_simplified":"democratie",
                    "segment":"ParlaMint-NL_2017-02-08-tweedekamer-10.u200"})

print("Got %d Hits:" % test_result['hits']['total']['value'])

for hit in test_result['hits']['hits']:
    print("%(person)s (%(party)s) \n(%(year)s-%(month)s-%(day)s) %(segment)s:\n %(content)s \n" % hit["_source"])

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Oude functie
# def get_file_name(file):
#     file = file.split('.')[0]+'.xml'
        
#     if 'FR' in file:
#         file = file.split('_u')[0]+'.xml'
    
#     return file

In [ ]:
def get_word_percentage(row):
    word_total = df_word_count[df_word_count["filename"] == row["file"]]["words"].values[0]
    return (row["percentage"] / word_total) * 100

In [ ]:
# neemt een query als input en geeft een df terug voor plotly express
# TODO: Standaardlanden fixen en zorgen dat de afweging voor zoeken met translation gemaakt kan worden
def get_ngram_df(query_input, timespan="years", countries=["NL", "BE", "FR"]):
    
    query_input_words = len(query_input["content_simplified"].split(" "))
    
    df = pd.DataFrame({'date': [],
                      'percentage': [],
                      'country': [],
                      'file': []})
    
    # doe een query per land
    for country in countries:
        query_input["country"] = country
        query_result = query(query_input)["hits"]["hits"]
    
        # loop door elk resultaat
        for hit in query_result:
            src = hit["_source"]

            # verschillende weergaven
            if timespan == "years":
                date = "{}-{}-{}".format(src["year"], 1, 1)
            elif timespan == "months":
                date = "{}-{}-{}".format(src["year"], src["month"], 1)
            else:
                date = "{}-{}-{}".format(src["year"], src["month"], src["day"])

            file = src["file"]

            df.loc[len(df.index)] = [date,
                                     (query_input_words * (src["content_simplified"].count(query_input["content_simplified"]))),
                                     country,
                                     file]

    # krijg het totaal aantal hits binnen 1 bestand van de zoekterm
    df["percentage"] = df.groupby("file")["percentage"].transform("sum")

    # schoon de df weer op
    df = df.drop_duplicates(["date", "percentage", "country", "file"])
    df = df.loc[df["percentage"] != 0]

    # reken per document het ngram percentage uit
    df["percentage"] = df.apply(get_word_percentage, axis=1)

    # voeg alle percentages op dezelfde dag samen
    df["percentage"] = df.groupby(["country", "date"])["percentage"].transform("sum")

    # ga elke date langs om het totale percentage te delen door het aantal files vand de dag
    countries = list(dict.fromkeys(df["country"].tolist()))

    # deel per land en date de percentage som door het aantal documenten op de datum
    for country in countries:
        dates = list(dict.fromkeys(df[df["country"] == country]["date"].tolist()))

        for date in dates:

            # deel het totale percentage door het totale aantal rows (van een date en country)
            count = len(df[(df["date"] == date) & (df["country"] == country)])
            df.update(df.loc[(df["country"] == country) & (df["date"] == date)]["percentage"].apply(lambda x: (x / count)))

    # gooi de dupes eruit
    df = df.drop_duplicates(["date", "country"]).drop("file", axis=1)
    
    return df.sort_values(by=["date"])

In [ ]:
start_time = time.time()

test_term = "democratie"

test_newgram = {"content_simplified":test_term}

test_ngram_df = get_ngram_df(test_newgram)

print("--- %s seconds ---" % (time.time() - start_time))
# display(test_ngram_df)

fig = px.line(test_ngram_df, x="date", y="percentage", color="country", title=f'Ngram: "{test_term}"')

fig.show()

In [ ]:
start_time = time.time()

test_term = "democratie"

test_newgram = {"content_simplified":test_term, "year":"2018"}

test_ngram_df = get_ngram_df(test_newgram, "months")

# display(test_ngram_df)

print("--- %s seconds ---" % (time.time() - start_time))

fig = px.line(test_ngram_df, x="date", y="percentage", color="country", title=f'Ngram: "{test_term}"')

fig.show()

In [ ]:
start_time = time.time()

test_term = "democratie"

test_newgram = {"content_simplified":test_term, "year":"2018", "month":"07"}

test_ngram_df = get_ngram_df(test_newgram, "days")

# display(test_ngram_df)

print("--- %s seconds ---" % (time.time() - start_time))

fig = px.line(test_ngram_df, x="date", y="percentage", color="country", title=f'Ngram: "{test_term}"')

fig.show()